In [ ]:
! sbatch_tail -p gpu -q gpu --gres=gpu:A100:4 --cpus-per-task 20 --mem-per-gpu 80G --ntasks-per-node=4 slurm/submit.sh "predict --config config/base_v3.yml --scprint_training.noise [0.7,1.0] --scprint_training.mask_ratio [] --model.dropout 0 --data.num_workers 17 --data.force_recompute_indices True --data.weight_scaler 0 --data.test_split 0 --data.batch_size 120 --trainer.num_nodes 1 --model.expr_emb_style metacell --data.get_knn_cells True --data.metacell_mode 0 --model.expr_encoder_layers 1 --model.compress_class_dim \"{'cell_type_ontology_term_id':64,'tissue_ontology_term_id':32,'disease_ontology_term_id':32,'age_group':8,'assay_ontology_term_id':12,'self_reported_ethnicity_ontology_term_id':8,'sex_ontology_term_id':2,'organism_ontology_term_id':8,'cell_culture':2}\" --model.layers_cls [256] --ckpt_path /pasteur/zeus/projets/p02/ml4ig_hot/Users/jkalfon/scprint_v2/czg34ita/checkpoints/last.ckpt"

In [ ]:
sbatch_tail -p gpu -q gpu --gres=gpu:1 --cpus-per-task 10 --mem-per-gpu 50G --ntasks-per-node=1 slurm/submit.sh "predict --config config/base_v3.yml --model.dropout 0 --data.num_workers 9 --data.force_recompute_indices True --data.weight_scaler 0 --data.test_split 0 --data.batch_size 60 --trainer.num_nodes 1 --model.expr_emb_style metacell --data.get_knn_cells True --data.metacell_mode 0 --model.expr_encoder_layers 1 --model.compress_class_dim \"{'cell_type_ontology_term_id':64,'tissue_ontology_term_id':32,'disease_ontology_term_id':32,'age_group':8,'assay_ontology_term_id':12,'self_reported_ethnicity_ontology_term_id':8,'sex_ontology_term_id':2,'organism_ontology_term_id':8,'cell_culture':2}\" --model.layers_cls [256] --ckpt_path /pasteur/zeus/projets/p02/ml4ig_hot/Users/jkalfon/scprint_v2/czg34ita/checkpoints/last.ckpt --data.store_location /pasteur/appa/scratch/jkalfon/predict_large"

## modified the model to replace logging of an anndata to logging of dataframe parquets


In [ ]:
# set this
self.keep_all_labels_pred = True

# replace with this

In [ ]:
cell_type_ontology_term_id':64,'tissue_ontology_term_id':32,'disease_ontology_term_id':32,'age_group':8,'assay_ontology_term_id':12,'self_reported_ethnicity_ontology_term_id':8,'sex_ontology_term_id':2,'organism_ontology_term_id':8,'cell_culture':2

In [ ]:
# know this:
model.classes
model.hparams["classes"](counts)
num = {
    "cell_type_ontology_term_id": 3,
    "tissue_ontology_term_id": 3,
    "disease_ontology_term_id": 3,
    "age_group": 3,
    "assay_ontology_term_id": 3,
    "self_reported_ethnicity_ontology_term_id": 3,
    "sex_ontology_term_id": 1,
    "organism_ontology_term_id": 3,
    "cell_culture": 3,
}

In [ ]:
def log_adata(self, gtclass=None, name=""):
    """
    log_adata will log an adata from predictions.
    It will log to tensorboard and wandb if available

    see @utils.log_adata
    """
    try:
        mdir = self.logger.save_dir if self.logger.save_dir is not None else "/tmp"
    except:
        mdir = "data/"
    if not os.path.exists(mdir):
        os.makedirs(mdir)
    self.embs = (
        torch.concat([v for k, v in self.embs.items()], dim=1)
        .cpu()
        .numpy()
        .astype(np.float16)
    )

    count = {
        "cell_type_ontology_term_id": 3,
        "tissue_ontology_term_id": 3,
        "disease_ontology_term_id": 3,
        "age_group": 3,
        "assay_ontology_term_id": 3,
        "self_reported_ethnicity_ontology_term_id": 3,
        "sex_ontology_term_id": 1,
        "organism_ontology_term_id": 3,
        "cell_culture": 1,
    }
    tot = 0
    locs = []
    predloc = []
    for val in self.classes:
        num = self.hparams["classes"][val]
        pred_slice = self.pred[:, tot : tot + num]
        loc = pred_slice.argsort(-1)[:, -count[val] :].flip([-1])
        top_k = torch.gather(pred_slice, 1, loc)
        predloc.append(top_k.cpu().numpy().astype(np.float16))
        locs.append(loc.cpu().numpy().astype(np.uint16))
        tot += num
    # this is a debugger line
    import pdb

    pdb.set_trace()
    predloc = np.hstack(predloc)
    locs = np.hstack(locs)
    save = (
        str(mdir)
        + "/step_"
        + str(self.global_step)
        + "_"
        + str(self.name)
        + "_"
        + str(name)
        + "_"
        + str(self.global_rank)
    )
    np.save(save + "_top3.npz", locs)
    np.save(save + "_scores.npz", predloc)
    np.save(save + "_embs.npz", self.embs)

In [2]:
import os
import numpy as np

In [10]:
embs = []
scores = []
cl=[]
LOC = '../../'
for i in os.listdir(LOC):
    if i.startswith('step_0__predict_part_') and i.endswith('_embs.npz.npy'):
        embs.append(np.load(LOC+i))
    if i.startswith('step_0__predict_part_') and i.endswith('_scores.npz.npy'):
        scores.append(np.load(LOC+i))
    if i.startswith('step_0__predict_part_') and i.endswith('_top3.npz.npy'):
        cl.append(np.load(LOC+i))
embs = np.vstack(embs)
scores = np.vstack(scores)
cl = np.vstack(cl)

In [16]:
cl.shape

(20040, 22)

In [ ]:
256 + 64 + 32 + 32 + 8 + 12 + 8 + 2 + 8 + 2

array([[402, 175, 416, ...,   0,   1,   1],
       [402, 175, 416, ...,   0,   1,   1],
       [402, 175, 416, ...,   0,   1,   1],
       ...,
       [402, 175, 416, ...,   0,   1,   1],
       [402, 175, 416, ...,   0,   1,   1],
       [402, 416, 175, ...,   0,   1,   1]],
      shape=(20040, 22), dtype=uint16)

In [17]:
cell_labels = cl[:, 0:3]

In [15]:
cell_emb = embs[:, 256:256+64]

In [19]:
from umap.parametric_umap import ParametricUMAP

/pasteur/appa/homes/jkalfon/scPRINT/.venv/lib/python3.11/site-packages/umap/parametric_umap.py:15: UserWarning: The umap.parametric_umap package requires Tensorflow > 2.0 to be installed.
    You can install Tensorflow at https://www.tensorflow.org/install
    
    or you can install the CPU version of Tensorflow using 

    pip install umap-learn[parametric_umap]

    
  warn(


ImportError: umap.parametric_umap requires Tensorflow >= 2.0

In [20]:
! uv pip install tensorflow[and-cuda]

Using Python 3.11.11 environment at: /pasteur/appa/homes/jkalfon/scPRINT/.venv
Resolved 50 packages in 1.04s                                        
⠙ Preparing packages... (0/18)                                                  
⠙ Preparing packages... (0/18)----     0 B/30.15 KiB                    
⠙ Preparing packages... (0/18)---- 14.87 KiB/30.15 KiB                  
astunparse ------------------------------     0 B/12.43 KiB
⠙ Preparing packages... (0/18)---- 14.87 KiB/30.15 KiB                  
astunparse ------------------------------ 12.43 KiB/12.43 KiB
⠙ Preparing packages... (0/18)---- 14.87 KiB/30.15 KiB                  
astunparse ------------------------------ 12.43 KiB/12.43 KiB
flatbuffers ------------------------------ 14.87 KiB/30.15 KiB
⠙ Preparing packages... (0/18)---     0 B/105.15 KiB                    
astunparse ------------------------------ 12.43 KiB/12.43 KiB
flatbuffers ------------------------------ 14.87 KiB/30.15 KiB
⠙ Preparing packages... (0/18)---

In [ ]:
embedder = ParametricUMAP(n_epochs=200, verbose=True)
embedder.fit_transform(cell_emb[:10_000])

NameError: name 'ParametricUMAP' is not defined

In [ ]:
fig, ax = plt.subplots()
ax.plot(embedder._history["loss"])
ax.set_ylabel("Cross Entropy")
ax.set_xlabel("Epoch")

In [ ]:
embedder.save("/tmp/model")

In [ ]:
from umap.parametric_umap import load_ParametricUMAP

embedder = load_ParametricUMAP("/tmp/model")

umap = embedder.transform(elems)
np.save(umap, "_umap.npz")